In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("steam_games.csv")

In [3]:
df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [4]:
# checking for missing data
df.isnull().sum()

url                             0
types                           2
name                           13
desc_snippet                 9742
recent_reviews              31931
all_reviews                  6414
release_date                 3018
developer                     249
publisher                    3294
popular_tags                 2897
game_details                  453
languages                      32
achievements                22811
genre                         355
game_description             2874
mature_content              32214
minimum_requirements        17121
recommended_requirements    17115
original_price               3425
discount_price              21419
dtype: int64

In [5]:
df.shape

(34633, 20)

In [6]:
selected_features = ["desc_snippet", "release_date", "developer", "publisher", "popular_tags", "name"]

In [7]:
selected_features

['desc_snippet',
 'release_date',
 'developer',
 'publisher',
 'popular_tags',
 'name']

In [8]:
# replace the null with ""
for feature in selected_features:
    df[feature] = df[feature].fillna("")

In [9]:
# Combine features:
combined_features = df["desc_snippet"]+" "+df["release_date"]+" "+df["developer"]+" "+df["publisher"]+" "+df["popular_tags"]+" "+df["name"]

In [10]:
combined_features

0        Now includes all three premium DLC packs (Unto...
1        PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...
2        Take command of your own mercenary outfit of '...
3        The post-soviet country of Chernarus is struck...
4        EVE Online is a community-driven spaceship MMO...
                               ...                        
34628     Feb 27, 2015 Allicorn Games KISS ltd,KISS ltd...
34629     Sep 13, 2018 La Moutarde Dear Villagers,Dear ...
34630     Jan 30, 2015 Bob Ginormocorp Holdings Ltd,Gin...
34631     Jan 20, 2015 Fancy Fish Games Fancy Fish Game...
34632                                                     
Length: 34633, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=15000,
    ngram_range=(1, 2),
    min_df=3
)

feature_vectors = vectorizer.fit_transform(combined_features)


In [12]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()


In [13]:
def recommend_games(game_name, top_n=10):
    close_match = difflib.get_close_matches(game_name, df['name'], n=1)

    if not close_match:
        return "Game not found"

    matched_name = close_match[0]
    idx = indices[matched_name]

  # checking for similarity for just 1 game
    sim_scores = cosine_similarity(
        feature_vectors[idx],
        feature_vectors
    ).flatten()

    similar_indices = sim_scores.argsort()[::-1][1:top_n+1]

    return df.loc[similar_indices, ['name', 'popular_tags', 'developer', 'publisher']]


In [14]:
recommend_games("elden ring")

,name,popular_tags,developer,publisher
10842,Fatal Twelve Mini Artbook,"Indie,Adventure,Casual",aiueoKompany,"Sekai Project,Sekai Project"
16586,Narcissu 10th Anniversary Anthology Project - ...,"Indie,Casual",stage-nana,"Sekai Project,Sekai Project"
11192,Heart of the Woods Original Soundtrack - Snowfall,"Indie,Casual",Studio Élan,"Sekai Project,Sekai Project"
11939,Rising Angels: Hope,"Adventure,Indie,Casual,Visual Novel,Anime,Fema...",IDHAS Studios,"Sekai Project,Sekai Project"
30968,Air - 2236 A.D. Original Soundtrack,"Indie,Casual,Nudity,Sexual Content",Chloro,"Sekai Project,Sekai Project"
13930,Narcissu 10th Anniversary Anthology Project - ...,"Indie,Casual",stage-nana,"Sekai Project,Sekai Project"
10695,National Park Girls - Original Soundtrack,"Indie,Casual",Studio Coattails,"Sekai Project,Sekai Project"
14478,Walkerman Act 2: Herdchaser-girl,"Indie,Gore,Violent,Nudity,Sexual Content",ScalemaiL,"Sekai Project,Sekai Project"
10844,Fatal Twelve Original Soundtrack,"Adventure,Indie,Casual",aiueoKompany,"Sekai Project,Sekai Project"
11191,Heart of the Woods Original Soundtrack - Moonl...,"Indie,Casual",Studio Élan,"Sekai Project,Sekai Project"


In [15]:
recommend_games("ghost")

,name,popular_tags,developer,publisher
6737,Race,"Racing,Casual,Indie,Simulation",RewindApp,"RewindApp,RewindApp"
29055,Kick The Puppet,"Casual,Action,Indie",RewindApp,"RewindApp,RewindApp"
25830,Obscurity,"Casual,Indie,Strategy",RewindApp,"RewindApp,RewindApp"
27657,Christmas Race 2,"Racing,Casual,Indie",RewindApp,"RewindApp,RewindApp"
27871,Knife Flipping,"Casual,Indie",RewindApp,"RewindApp,RewindApp"
10868,Geometry Rush,"Casual,Indie",RewindApp,"RewindApp,RewindApp"
18748,Christmas Race,"Indie,Casual",RewindApp,"RewindApp,RewindApp"
21741,MineSweep,"Casual,Indie,Strategy",RewindApp,"RewindApp,RewindApp"
27769,Kings,"Casual,Strategy,Indie",RewindApp,"RewindApp,RewindApp"
27612,Real Drift,"Simulation,Indie,Casual,Racing","FRAdrift,RewindApp","RewindApp,RewindApp"
